In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Tuple
from preprocessing import (
    clsf_boz_cd, clsf_rrnr_dvcd, clsf_dmfr_dvcd, clsf_crd_grd, clsf_pdgr_cd,
    get_cf, get_disc_factor, get_loss_adj
)

In [2]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('./data/현행추정부채_일반')

In [3]:
# 데이터 불러오기
## 익스포져
일반_원수_미경과보험료 = pd.read_excel(FILE_PATH / '일반_원수_미경과보험료.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str})
일반_출재_미경과보험료 = pd.read_excel(FILE_PATH / '일반_출재_미경과보험료.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})
일반_원수_예정보험료 = pd.read_excel(FILE_PATH / '일반_원수_예정보험료.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str})
일반_출재_예정보험료 = pd.read_excel(FILE_PATH / '일반_출재_예정보험료.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})
일반_원수_개별추산액 = pd.read_excel(FILE_PATH / '일반_원수_개별추산액.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'ARC_INPL_CD': str})
일반_출재_개별추산액 = pd.read_excel(FILE_PATH / '일반_출재_개별추산액.xlsx', dtype={'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'ARC_INPL_CD': str, 'T02_RN_RINSC_CD': str})

## 기초정보, 가정 등
일반_상품정보 = pd.read_excel(FILE_PATH / '일반_상품정보.xlsx', dtype={'PDC_CD': str, 'PDGR_CD': str})
일반_보험금진전추이 = pd.read_excel(FILE_PATH / '일반_보험금진전추이.xlsx')
일반_최종손해율 = pd.read_excel(FILE_PATH / '일반_최종손해율.xlsx', dtype={'RRNR_DAT_DVCD': str, 'PDGR_CD': str}) \
        .rename(columns= {'RRNR_DAT_DVCD': 'RRNR_DVCD'})

## 공통
국가구분 = pd.read_excel(FILE_PATH / '국가구분.xlsx', dtype={'CNTR_CATG_CD': str})
재보험자_국내신용등급 = pd.read_excel(FILE_PATH / '재보험자_국내신용등급.xlsx', dtype={'재보험사코드': str}) \
        .rename(columns = {'재보험사코드': 'T02_RN_RINSC_CD', '국내신용등급': 'CRD_GRD'})
선도부도율 = pd.read_excel(FILE_PATH / '선도부도율.xlsx')
할인율 = pd.read_excel(FILE_PATH / '할인율.xlsx')

In [4]:
# 데이터 전처리
일반_원수_미경과보험료['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_미경과보험료, '원수')
일반_원수_미경과보험료['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_미경과보험료)
일반_원수_미경과보험료['BOZ_CD'] = clsf_boz_cd(일반_원수_미경과보험료, 일반_상품정보)
일반_원수_미경과보험료['PDGR_CD'] = clsf_pdgr_cd(일반_원수_미경과보험료, 일반_상품정보)

일반_출재_미경과보험료['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_출재_미경과보험료, '출재')
일반_출재_미경과보험료['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_미경과보험료)
일반_출재_미경과보험료['BOZ_CD'] = clsf_boz_cd(일반_출재_미경과보험료, 일반_상품정보)
일반_출재_미경과보험료['PDGR_CD'] = clsf_pdgr_cd(일반_출재_미경과보험료, 일반_상품정보)
일반_출재_미경과보험료['CRD_GRD'] = clsf_crd_grd(일반_출재_미경과보험료, 재보험자_국내신용등급)

일반_원수_예정보험료['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_예정보험료, '원수')
일반_원수_예정보험료['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_예정보험료)
일반_원수_예정보험료['BOZ_CD'] = clsf_boz_cd(일반_원수_예정보험료, 일반_상품정보)
일반_원수_예정보험료['PDGR_CD'] = clsf_pdgr_cd(일반_원수_예정보험료, 일반_상품정보)

일반_출재_예정보험료['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_출재_예정보험료, '출재')
일반_출재_예정보험료['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_예정보험료)
일반_출재_예정보험료['BOZ_CD'] = clsf_boz_cd(일반_출재_예정보험료, 일반_상품정보)
일반_출재_예정보험료['PDGR_CD'] = clsf_pdgr_cd(일반_출재_예정보험료, 일반_상품정보)
일반_출재_예정보험료['CRD_GRD'] = clsf_crd_grd(일반_출재_예정보험료, 재보험자_국내신용등급)

일반_원수_개별추산액['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_개별추산액, '원수')
일반_원수_개별추산액['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_개별추산액)
일반_원수_개별추산액['BOZ_CD'] = clsf_boz_cd(일반_원수_개별추산액, 일반_상품정보)
일반_원수_개별추산액['PDGR_CD'] = clsf_pdgr_cd(일반_원수_개별추산액, 일반_상품정보)

일반_출재_개별추산액['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_출재_개별추산액, '출재')
일반_출재_개별추산액['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_개별추산액)
일반_출재_개별추산액['BOZ_CD'] = clsf_boz_cd(일반_출재_개별추산액, 일반_상품정보)
일반_출재_개별추산액['PDGR_CD'] = clsf_pdgr_cd(일반_출재_개별추산액, 일반_상품정보)
일반_출재_개별추산액['CRD_GRD'] = clsf_crd_grd(일반_출재_개별추산액, 재보험자_국내신용등급)

In [50]:
# 데이터 집계
## 미경과보험료
일반_원수_미경과보험료_집계 = 일반_원수_미경과보험료.copy()
일반_원수_미경과보험료_집계['RRNR_DVCD'] = '04'
일반_원수_미경과보험료_집계.loc[lambda x: x['DMFR_DVCD'] == '02', 'BOZ_CD'] = '#'
일반_원수_미경과보험료_집계 = 일반_원수_미경과보험료_집계.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD'])['LTPD_URND_PRM'].sum().reset_index()
일반_원수_미경과보험료_집계 = 일반_원수_미경과보험료_집계.rename(columns = {'LTPD_URND_PRM': 'URND_PRM'})

일반_출재_미경과보험료_집계 = 일반_출재_미경과보험료.copy()
일반_출재_미경과보험료_집계.loc[lambda x: x['DMFR_DVCD'] == '02', 'BOZ_CD'] = '#'
일반_출재_미경과보험료_집계 = 일반_출재_미경과보험료_집계.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD'])['T02_LTPD_RN_URND_PRM'].sum().reset_index()
일반_출재_미경과보험료_집계 = 일반_출재_미경과보험료_집계.rename(columns = {'T02_LTPD_RN_URND_PRM': 'URND_PRM'})

일반_미경과보험료_집계 = pd.concat([일반_원수_미경과보험료_집계, 일반_출재_미경과보험료_집계], axis=0).reset_index(drop=True)

## 예정보험료
일반_원수_예정보험료_집계 = 일반_원수_예정보험료.copy()
일반_원수_예정보험료_집계['RRNR_DVCD'] = '04'
일반_원수_예정보험료_집계.loc[lambda x: x['DMFR_DVCD'] == '02', 'BOZ_CD'] = '#'
일반_원수_예정보험료_집계 = 일반_원수_예정보험료_집계.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD'])['WNCR_CNV_EPCT_PRM'].sum().reset_index()
일반_원수_예정보험료_집계 = 일반_원수_예정보험료_집계.rename(columns = {'WNCR_CNV_EPCT_PRM': 'EPCT_PRM'})

일반_출재_예정보험료_집계 = 일반_출재_예정보험료.copy()
일반_출재_예정보험료_집계.loc[lambda x: x['DMFR_DVCD'] == '02', 'BOZ_CD'] = '#'
일반_출재_예정보험료_집계 = 일반_출재_예정보험료_집계.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD'])['WNCR_CNV_RN_PRM'].sum().reset_index()
일반_출재_예정보험료_집계 = 일반_출재_예정보험료_집계.rename(columns = {'WNCR_CNV_RN_PRM': 'EPCT_PRM'})

일반_예정보험료_집계 = pd.concat([일반_원수_예정보험료_집계, 일반_출재_예정보험료_집계], axis=0).reset_index(drop=True)

일반_보험료익스포져 = 일반_미경과보험료_집계.merge(일반_예정보험료_집계, on=['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD'], how='outer') \
        .assign(URND_PRM = lambda x: x['URND_PRM'].fillna(0).astype(float)) \
        .assign(EPCT_PRM = lambda x: x['EPCT_PRM'].fillna(0).astype(float))
일반_보험료익스포져

,RRNR_DVCD,DMFR_DVCD,BOZ_CD,URND_PRM,EPCT_PRM
0,04,01,A001,"23,420,916,171",0
1,04,01,A002,"27,599,989,968","1,655,668,169"
2,04,01,A003,"96,919,532,915","3,881,525,083"
3,04,01,A004,"23,093,930,741","12,982,384,892"
4,04,01,A005,"5,488,832,989","1,000,300,401"
5,04,01,A006,"66,551,730,700","3,223,443,768"
6,04,01,A007,"93,999,089,721","4,971,387,805"
7,04,01,A008,"4,773,218,318",0
8,04,01,A009,"8,675,613,167",0
9,04,01,A010,"75,028,541,789","8,993,019"


In [7]:
# 데이터 내보내기


,RRNR_DVCD,DMFR_DVCD,BOZ_CD,LTPD_URND_PRM
0,01,01,A001,23420916171
1,01,01,A002,27146263273
2,01,01,A003,96625029260
3,01,01,A004,22191701960
4,01,01,A005,5095840261
5,01,01,A006,66353121310
6,01,01,A007,93981583216
7,01,01,A008,4773218318
8,01,01,A010,73720023337
9,01,01,A011,3516025
